# eventchain

> Building and anylizing event chains in a convenient way.

In [ ]:
#| default_exp core

In [ ]:
#| export
import pandas as pd
from scipy import stats
from scipy.stats import norm

In [ ]:
#| export

class EventChain:
    
    def __init__(self, df1: pd.DataFrame, df2: pd.DataFrame, user_col: str, timestamp_col: str, suffix):
        
        self.timestamp_a = f"{timestamp_col}_{suffix[0]}"
        self.timestamp_b = f"{timestamp_col}_{suffix[1]}"

        self.user_col = user_col    
        self.any_event_A = df1.rename(columns={timestamp_col: self.timestamp_a})
        self.any_event_B = df2.rename(columns={timestamp_col: self.timestamp_b})
        self.first_event_A = self.any_event_A.groupby(self.user_col)[f"{timestamp_col}_{suffix[0]}"].min().reset_index()
        self.last_event_A = self.any_event_A.groupby(self.user_col)[f"{timestamp_col}_{suffix[0]}"].max().reset_index()
        self.first_event_B = self.any_event_B.groupby(self.user_col)[f"{timestamp_col}_{suffix[1]}"].min().reset_index()
        self.last_event_B = self.any_event_B.groupby(self.user_col)[f"{timestamp_col}_{suffix[1]}"].max().reset_index()
            
    def anti_join(self, df1, df2): 
        return df1.merge(df2, how="left", on=self.user_col, indicator=True).query("_merge == 'left_only'").drop(columns="_merge")

    def before(self, df1, df2): 
        return (df1
            .merge(df2, how="inner", on=self.user_col).query(f"{self.timestamp_a} < {self.timestamp_b}")
            .assign(gap=lambda x: x[self.timestamp_b] - x[self.timestamp_a]))

    def before_or_equal(self, df1, df2): 
        return (df1
            .merge(df2, how="inner", on=self.user_col).query(f"{self.timestamp_a} <= {self.timestamp_b}")
            .assign(gap=lambda x: x[self.timestamp_b] - x[self.timestamp_a]))

    @property
    def first_before_first(self): 
        return self.before(self.first_event_A, self.first_event_B)

    @property
    def first_before_or_equal_first(self):
        return self.before_or_equal(self.first_event_A, self.first_event_B)
    
    @property
    def first_before_last(self): 
        return self.before(self.first_event_A, self.last_event_B)

    @property
    def first_before_or_equal_last(self): 
        return self.before_or_equal(self.first_event_A, self.last_event_B)
    
    @property
    def last_before_first(self):
        return self.before(self.last_event_A, self.first_event_B)

    @property
    def last_before_or_equal_first(self):
        return self.before_or_equal(self.last_event_A, self.first_event_B)

    @property
    def last_before_last(self):
        return self.before(self.last_event_A, self.last_event_B)

    @property
    def last_before_or_equal_last(self):
        return self.before_or_equal(self.last_event_A, self.last_event_B)

    @property
    def any_before_any(self):
        return self.before_or_equal(self.any_event_A, self.any_event_B)

    @property
    def any_before_first(self):
        return self.before_or_equal(self.any_event_A, self.first_event_B)

    @property
    def any_before_or_equal_first(self):
        return self.before_or_equal(self.any_event_A, self.first_event_B)

    @property
    def any_before_last(self):
        return self.before_or_equal(self.any_event_A, self.last_event_B)

    @property
    def any_before_or_equal_last(self):
        return self.before_or_equal(self.any_event_A, self.last_event_B)

    @property
    def first_before_none(self): 
        return self.anti_join(self.first_event_A, self.any_event_B)
    
    @property
    def last_before_none(self): 
        return self.anti_join(self.last_event_A, self.any_event_B)
    
    @property
    def none_before_first(self): 
        return self.anti_join(self.first_event_B, self.any_event_A)
    
    @property
    def none_before_last(self): 
        return self.anti_join(self.last_event_B, self.any_event_A)

In [ ]:
#| export

def get_stats(nominator_df, denominator_df, user_col, gap_column='gap'):
    nominator_df[gap_column] = nominator_df[gap_column].dt.days
    nominator = nominator_df[user_col].nunique()
    denominator = denominator_df[user_col].nunique()
    n = len(nominator_df)
    
    summary_stats = {}
    summary_stats['no_events_of_interest'] = nominator
    summary_stats['all_events'] = denominator
    summary_stats['pct'] = (nominator / denominator) * 100
    summary_stats['mean_gap'] = nominator_df[gap_column].mean()
    summary_stats['median_gap'] = nominator_df[gap_column].median()
    summary_stats['std_dev'] = nominator_df[gap_column].std()
    summary_stats['min_gap'] = nominator_df[gap_column].min()
    summary_stats['max_gap'] = nominator_df[gap_column].max()
    summary_stats['25th_percentile'] = nominator_df[gap_column].quantile(0.25)
    summary_stats['50th_percentile'] = nominator_df[gap_column].quantile(0.5)
    summary_stats['75th_percentile'] = nominator_df[gap_column].quantile(0.75)
    summary_stats['skewness'] = nominator_df[gap_column].skew()
    summary_stats['kurtosis'] = nominator_df[gap_column].kurt()
    
    confidence_level = 0.95
    z_critical = norm.ppf(1 - (1 - confidence_level) / 2)
    margin_of_error = z_critical * (summary_stats['std_dev'] / (n ** 0.5))
    summary_stats['conf_interval_lower'] = summary_stats['mean_gap'] - margin_of_error
    summary_stats['conf_interval_upper'] = summary_stats['mean_gap'] + margin_of_error

    print("="*50)
    print("{:<30} {:<10}".format('Metric', 'Value'))
    print("="*50)

    for key, value in summary_stats.items():
        print("{:<30} {:<10}".format(key, value))

    print("="*50)

    return summary_stats

In [ ]:
#| hide
shoppingcard = pd.read_csv('../data/visited.csv')
purchased = pd.read_csv('../data/purchased.csv')

shoppingcard['date'] = pd.to_datetime(shoppingcard['date'])
purchased['date'] = pd.to_datetime(purchased['date'])

In [ ]:
#| hide
chain = EventChain(df1=shoppingcard, df2=purchased, user_col="user_id", timestamp_col="date", suffix=["added_to_shoppingcard", "purchased"])

In [ ]:
#| hide
stats = get_stats(chain.first_before_first, chain.any_event_A, user_col="user_id", gap_column="gap")

Metric                         Value     
no_events_of_interest          4         
all_events                     7         
pct                            57.14285714285714
mean_gap                       1.5       
median_gap                     1.0       
std_dev                        1.0       
min_gap                        1         
max_gap                        3         
25th_percentile                1.0       
50th_percentile                1.0       
75th_percentile                1.5       
skewness                       1.9999999999999998
kurtosis                       4.0       
conf_interval_lower            0.520018007729973
conf_interval_upper            2.479981992270027


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()